In [10]:
import tweepy #This is the python Twitter API
from textblob.classifiers import NaiveBayesClassifier #This is the model that was used to classify tweets
import re #python regular expressions

In [11]:
#This aouthentication stuff is used to get tweets

consumer_key = "idrlfeZz3xKG2ZIljCMupqGwn"
consumer_secret = "7qpiiNlqxTyXkujKRaxd1esHyr3Xthydavarx05gSgNGEUnuAY"

access_token = "969724922124816384-9vBm7uhnH9XpLLxFvO34TtOdYeczg41"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

api = tweepy.API(auth)

'public_tweets = api.home_timeline()\nfor tweet in public_tweets:\n    print(tweet.text)'

In [24]:
def get_tweets_lean(tag,leaning,count_):
    '''
    This function takes a twitter user then scrapes and cleanes their most recent tweets and tags 
    them all with a political leaning which is known beforehand
    '''
    try:
        # Get the person's tweets
        stuff = api.user_timeline(screen_name = tag, count = count_,include_rts = False)
    except:
        print("error when looking for " + tag + "'s tweets")
        return([])

    # Return a list of (tweet,tag) tuple for all of the tweets, this assumes that every tweet from
    # the person in questional is tweet with their political leaning
    return [(cleantweet(tweet.text.encode("utf-8")),leaning) for tweet in stuff]

def get_tweets(tag,count_):
    '''
    This function takes a twitter user then scrapes and cleanes their most recent tweets without 
    tagging them
    '''
    try:
        stuff = api.user_timeline(screen_name = tag, count = count_, include_rts = False)
    except:
        print("error when looking for " + tag + "'s tweets")
        return([])

    return [cleantweet(tweet.text.encode("utf-8")) for tweet in stuff]

In [13]:
def cleantweet(x):
    #Regular expression to clean any weird characters from the tweets
    x = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x.decode("utf-8"))
    return(x)

In [25]:
#Here we decide how many tweets to get from every person
numTweets = 50

#We manually picked a list of liberal and conservative influencers to train the model, liberals were
#assigned the negative label and conservatives the positive label (this decision was arbitrary)
liberals = [("SaraJBenincasa", "neg"), ("DrJaneChi", "neg"), ("caulkthewagon", "neg"),("melissagira", "neg"), ("AngryBlackLady","neg"),("andreagrimes","neg"),("ClinicEscort","neg"),("katie_speak","neg"),("TheChrisMosier","neg"),("mollyknefel","neg"),("scATX", "neg"),("gaywonk","neg"),("jamiekilstein","neg"),("pastachips","neg"),("TransGriot","neg"),("chrisgeidner","neg"),("CameronCRussell","neg"),("lsarsour","neg"),("ztsamudzi","neg")]
conservatives = [("MichaelJohns","pos"),("JohnBoehner","pos"),("SarahPalinUSA","pos"),("Heritage","pos"),("RedState","pos"),("GlennBeck","pos"),("KarlRove","pos"),("NewtGingrich","pos"),("FredThompson","pos"),("MittRomney","pos"),("JoeNBC","pos"),("SeanHannity","pos"),("theMRC", "pos"),("RNC","pos"),("dickmorristweet","pos"),("LoyaltoLiberty","pos"),("HotAirBlog","pos"),("USConservatives","pos")]

allHandles = liberals + conservatives
data = []

for handle in allHandles:
    for thing in get_tweets_lean(handle[0],handle[1],numTweets):
        #every individual (tweet,leaning) tuple from every influencer was put into the list data
        data.append(thing)

error when looking for JohnBoehner's tweets


In [26]:
#here we define and train the classifier
classifier = NaiveBayesClassifier(data)

In [21]:
#this gives a twitter a score based on how many of their tweets are classified as conservative
def predict(user):
    tweets = get_tweets(user,numTweets)
    scores = [classifier.classify(tweet) for tweet in tweets]
    return (scores.count("pos")/len(scores))

In [29]:
#inputdata is a list of senators which we were evaluated on (this list was not available until 
#the end of the challenge)
inputData = ['SenDougJones', 'SenTomCotton', 'SenatorRounds', 'SenFeinstein', 'SenatorHeitkamp', 'SenSasse', 'SenatorRisch', 'SenJackReed', 'GOPSenFinance', 'SenTedCruz', 'BankingGOP', 'SenatorTester', 'SenBrianSchatz', 'SenatorFischer', 'SenJoniErnst', 'HELPCmteDems', 'SASCDems', 'SenJohnKennedy', 'SenateMajLdr', 'SenTinaSmith', 'SenateDems', 'SenDanSullivan', 'SenKamalaHarris', 'MikeCrapo', 'SenateCommerce', 'sendavidperdue', 'SenatorHassan', 'SenateHSGACDems', 'senategopfloor', 'EPWGOP', 'SenAngusKing', 'SenatorBaldwin', 'SenCortezMasto', 'SenThadCochran', 'SenThomTillis', 'SenJohnHoeven', 'SenDuckworth', 'MartinHeinrich', 'SenWarren', 'SenateGOP', 'SenBooker', 'JudiciaryDems']
results = [(senator,predict(senator)) for senator in inputData]

In [30]:
results

[('SenDougJones', 0.3333333333333333),
 ('SenTomCotton', 0.825),
 ('SenatorRounds', 0.8260869565217391),
 ('SenFeinstein', 0.7291666666666666),
 ('SenatorHeitkamp', 0.5416666666666666),
 ('SenSasse', 0.5416666666666666),
 ('SenatorRisch', 0.6451612903225806),
 ('SenJackReed', 0.6944444444444444),
 ('GOPSenFinance', 0.7647058823529411),
 ('SenTedCruz', 0.8260869565217391),
 ('BankingGOP', 0.85),
 ('SenatorTester', 0.5609756097560976),
 ('SenBrianSchatz', 0.7),
 ('SenatorFischer', 0.4634146341463415),
 ('SenJoniErnst', 0.6530612244897959),
 ('HELPCmteDems', 0.6),
 ('SASCDems', 0.6896551724137931),
 ('SenJohnKennedy', 0.6176470588235294),
 ('SenateMajLdr', 0.8333333333333334),
 ('SenTinaSmith', 0.2653061224489796),
 ('SenateDems', 0.6),
 ('SenDanSullivan', 0.5306122448979592),
 ('SenKamalaHarris', 0.6326530612244898),
 ('MikeCrapo', 0.7037037037037037),
 ('SenateCommerce', 0.8421052631578947),
 ('sendavidperdue', 0.6486486486486487),
 ('SenatorHassan', 0.5625),
 ('SenateHSGACDems', 0.6),
